In [1]:
import pandas as pd
import re
from collections import defaultdict
import itertools
import numpy as np

In [2]:
df = pd.read_csv('./dados/respostas.csv')

In [3]:
def renomear_colunas(df):
    REGEX_DESCRICAO_COLUNA = re.compile('.+\[(.+)\]')
    
    colunas = {
        'Qual a sua faixa etária?': 'faixa etária',
        'Qual a sua escolaridade?': 'escolaridade',
        'Qual a sua profissão atual?': 'profissão',
        'Existe alguma ferramenta/tecnologia/linguagem que não foi mencionada na pergunta anterior?': 'outras ferramentas',
        'Em qual dia/horário você teria disponibilidade para participar dos eventos (palestras/workshops) do Hackerspace?': 'disponibilidade'
    }

    for coluna in df.columns:
        match = REGEX_DESCRICAO_COLUNA.search(coluna)
        if match:
            colunas[coluna] = match.group(1).lower()

    return df.rename(columns=colunas)

In [4]:
def limpar_respostas(df):
    df = df.fillna('')
    return df.replace({
        'Utilizo na EMPRESA': 'EMPRESA',
        'Tenho INTERESSE em aprender': 'INTERESSE',
        'Utilizo na EMPRESA, Tenho INTERESSE em aprender': 'EMPRESA, INTERESSE',
    })

In [5]:
df = renomear_colunas(df)
df = limpar_respostas(df)
df.head()

,Timestamp,faixa etária,escolaridade,profissão,bash/shell/power shell,c/c++,c#,delphi,go,java,...,"desenvolvimento ágil (kanban, scrum, xp)",jira,trello,git,svn,itil,infraestrutura,redes de computadores,outras ferramentas,disponibilidade
0,1/25/2019 8:29:17,26 a 35 anos,Cursando Graduação,Programador/Desenvolvedor,,,,,,EMPRESA,...,,,,,,,,,,Sábado de manhã (9h às 12h)
1,1/25/2019 8:30:25,19 a 25 anos,Cursando Graduação,Programador/Desenvolvedor,,,,EMPRESA,,EMPRESA,...,,,,,,,,,,Sábado de manhã (9h às 12h)
2,1/25/2019 9:00:08,36 a 45 anos,Pós-graduação Completa,Gerente de Projetos/Produto,,,,,,EMPRESA,...,INTERESSE,,,INTERESSE,,,,,,Sábado de manhã (9h às 12h)
3,1/25/2019 9:13:45,36 a 45 anos,Pós-graduação Completa,Analista de Sistemas,,,,,,,...,INTERESSE,,INTERESSE,INTERESSE,,,,,,"Quinta-feira à noite (19h às 22h), Sábado de m..."
4,1/25/2019 9:27:34,26 a 35 anos,Pós-graduação Completa,Programador/Desenvolvedor,"EMPRESA, INTERESSE",,INTERESSE,,INTERESSE,,...,"EMPRESA, INTERESSE",EMPRESA,EMPRESA,"EMPRESA, INTERESSE",,,,,Interesse: Data science/Machine learning; TDD;...,"Quinta-feira à noite (19h às 22h), Sábado de m..."


In [6]:
profissoes = {
    'Programador/Cloud/DevOps': 'Programador/Desenvolvedor',
    'Software Engineer': 'Programador/Desenvolvedor',
    'Arquiteto de soluções': 'Arquiteto de software',
    'Arquiteto de Software': 'Arquiteto de software',
    'Arquiteto': 'Arquiteto de software',
    'Estagiário / Testes': 'Estagiário',
    'estagiario': 'Estagiário',
    'Coordenador de TI': 'Coordenador de Desenvolvimento',
    'Não trabalho na area': 'Não trabalho na área'
}

def normalizar_profissao(profissao):
    profissao = profissao.strip()
    return profissoes.get(profissao, profissao)

df['profissão'] = df['profissão'].apply(normalizar_profissao)

In [7]:
escolaridades = {
    'Técnico Completo e cursando o Superior': 'Cursando Graduação',
    'Mestrado completo': 'Pós-graduação Completa'
}

def normalizar_escolaridade(escolaridade):
    escolaridade = escolaridade.strip()
    return escolaridades.get(escolaridade, escolaridade)

df['escolaridade'] = df['escolaridade'].apply(normalizar_escolaridade)

In [8]:
df['faixa etária'] = df['faixa etária'].str.replace(' a ', ' à ')

In [9]:
def padrozinar_pontuacao(texto):
    return re.sub('\s*[/:,&.]\s*', '; ', texto)
    
def padrozinar_conjuncao_e(texto):
    return re.sub('\s+e\s+', '; ', texto)

def remover_aspas(texto):
    return texto.replace('"', '; ')

def remover_quebra_linha(texto):
    return texto.replace('\r', '').replace('\n', '; ')

def lower(texto):
    return texto.lower()

def apply(series, funcs):
    for f in funcs:
        series = series.apply(f)
        
    return series

funcs = [padrozinar_pontuacao, padrozinar_conjuncao_e, remover_aspas, remover_quebra_linha, lower]
df['outras ferramentas padrao'] = apply(df['outras ferramentas'], funcs)

In [10]:
def split(texto):
    return texto.split(';')

def strip(texto):
    return texto.strip()

def remover_pontuacao(texto):
    return re.sub('[^\w\s\d]|\r\n', ' ', texto)

def tokens(texto):
    return [strip(remover_pontuacao(t)) for t in split(texto)]

def padronizar_token(token):
    TOKENS_PADRONIZADOS = {
        'interesse em aprender mais sobre': '',
        'ezok': '',
        'js npm': 'node',
        'micro serviçois': 'micro serviços',
        'devops basic': 'devops',
        'ides como webstorm': 'webstorm',
        'afins': '',
        'elastic search': 'elk',
        'elasticsearch': 'elk',
        'carga': 'teste de carga',
        'stress': 'teste de stress',
        'testes tdd': 'tdd',
        'bitcoin': 'criptomoeda',
        'vs code': 'vscode'
    }
    
    if 'amazon' in token or 'aws' in token:
        return 'aws'
    elif 'ruby' in token:
        return 'ruby'
    
    return TOKENS_PADRONIZADOS.get(token, token)

def agrupar_tokens(tipo):
    
    def agrupar(texto):
        grupos = {
            'empresa': [],
            'interesse': []
        }

        lista = grupos['interesse']
        for token in tokens(texto):
            token = padronizar_token(token)
            if not token:
                continue

            if token in grupos:
                lista = grupos[token]
                continue

            lista.append(token)
            
        return ';'.join(grupos[tipo])
    
    return agrupar

df['interesse'] = df['outras ferramentas padrao'].apply(agrupar_tokens('interesse'))
df['empresa'] = df['outras ferramentas padrao'].apply(agrupar_tokens('empresa'))

In [11]:
colunas = [
    'bash/shell/power shell', 'c/c++', 'c#', 'delphi', 'go', 'java',
    'javascript', 'php', 'python', 'banco de dados relacional (sql)',
    'banco de dados não relacional (no sql)', 'selenium', 'test complete',
    'enterprise architect', 'desenvolvimento ágil (kanban, scrum, xp)',
    'jira', 'trello', 'git', 'svn', 'itil', 'infraestrutura',
    'redes de computadores'
]

renomear = {
    'bash/shell/power shell': 'terminal',
    'banco de dados relacional (sql)': 'sql',
    'banco de dados não relacional (no sql)': 'no sql',
    'desenvolvimento ágil (kanban, scrum, xp)': 'desenvolvimento ágil',
}

def adicionar_token(token, tokens, adicionar):
    if adicionar:
        return tokens + ';' + token
    
    return tokens

for coluna in colunas:
    token = renomear.get(coluna, coluna)
    series = pd.Series([''] * len(df))
    series[df[coluna].str.contains('INTERESSE')] = ';' + token
    df['interesse'] += series
    
    series = pd.Series([''] * len(df))
    series[df[coluna].str.contains('EMPRESA')] = ';' + token
    df['empresa'] += series

In [12]:
df.interesse = df.interesse.apply(lambda s: s[1:] if s and s[0] == ';' else s)
df.empresa = df.empresa.apply(lambda s: s[1:] if s and s[0] == ';' else s)

In [13]:
df = df.drop(colunas + ['outras ferramentas', 'outras ferramentas padrao'], 1)
df.columns

Index(['Timestamp', 'faixa etária', 'escolaridade', 'profissão',
       'disponibilidade', 'interesse', 'empresa'],
      dtype='object')

In [14]:
df.to_csv('./dados/normalizado.csv', index=False)